In [7]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("psparks/instacart-market-basket-analysis")

print("Path to dataset files:", path)


Path to dataset files: C:\Users\ROG STRIX\.cache\kagglehub\datasets\psparks\instacart-market-basket-analysis\versions\1


In [10]:
import pandas as pd
import numpy as np

In [3]:
aisles = pd.read_csv(path + "/aisles.csv")
departments = pd.read_csv(path + "/departments.csv")
orders = pd.read_csv(path + "/orders.csv")
order_products__prior = pd.read_csv(path + "/order_products__prior.csv")
order_products__train = pd.read_csv(path + "/order_products__train.csv")
products = pd.read_csv(path + "/products.csv")

In [8]:
print("shape of aisles:",aisles.shape)
print("shape of departments:",departments.shape)
print("shape of order_products__prior:",order_products__prior.shape)
print("shape of order_products__train:",order_products__train.shape)
print("shape of orders:",orders.shape)
print("shape of products:",products.shape)

print("aisles exploration:")
print(aisles.head())
print(aisles.tail())
print(aisles.info())
print(aisles.describe())
print(aisles.corr(numeric_only=True))

print("departments exploration:")
print(departments.head())
print(departments.tail())
print(departments.info())
print(departments.describe())
print(departments.corr(numeric_only=True))

print("order_products__prior exploration:")
print(order_products__prior.head())
print(order_products__prior.tail())
print(order_products__prior.info())
print(order_products__prior.describe())
print(order_products__prior.corr())

print("order_products__train exploration:")
print(order_products__train.head())
print(order_products__train.tail())
print(order_products__train.info())
print(order_products__train.describe())
print(order_products__train.corr())

print("orders exploration:")
print(orders.head())
print(orders.tail())
print(orders.info())
print(orders.describe())
print(orders.corr(numeric_only=True))
print(orders.eval_set.tail())
print(orders.order_hour_of_day)

print("products exploration:")
print(products.head())
print(products.tail())
print(products.info())
print(products.describe())
print(products.corr(numeric_only=True))

shape of aisles: (134, 2)
shape of departments: (21, 2)
shape of order_products__prior: (32434489, 4)
shape of order_products__train: (1384617, 4)
shape of orders: (3421083, 7)
shape of products: (49688, 4)
aisles exploration:
   aisle_id                       aisle
0         1       prepared soups salads
1         2           specialty cheeses
2         3         energy granola bars
3         4               instant foods
4         5  marinades meat preparation
     aisle_id                       aisle
129       130    hot cereal pancake mixes
130       131                   dry pasta
131       132                      beauty
132       133  muscles joints pain relief
133       134  specialty wines champagnes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 134 entries, 0 to 133
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   aisle_id  134 non-null    int64 
 1   aisle     134 non-null    object
dtypes: int64(1), object(

In [14]:
all_product= pd.concat([order_products__prior,order_products__train])
all_product.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [21]:
X=(products.merge(aisles,on='aisle_id').merge(departments,on='department_id'))
X

,product_id,product_name,aisle_id,department_id,aisle,department
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks
1,2,All-Seasons Salt,104,13,spices seasonings,pantry
2,3,Robust Golden Unsweetened Oolong Tea,94,7,tea,beverages
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,frozen meals,frozen
4,5,Green Chile Anytime Sauce,5,13,marinades meat preparation,pantry
...,...,...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5,spirits,alcohol
49684,49685,En Croute Roast Hazelnut Cranberry,42,1,frozen vegan vegetarian,frozen
49685,49686,Artisan Baguette,112,3,bread,bakery
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8,cat food care,pets


In [27]:
all=\
orders.merge(all_product,on='order_id')
all=\
X.merge(all,on='product_id')
all

,product_id,product_name,aisle_id,department_id,aisle,department,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,add_to_cart_order,reordered
0,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,3139998,138,prior,28,6,11,3.0,5,0
1,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,1977647,138,prior,30,6,17,20.0,1,1
2,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,389851,709,prior,2,0,21,6.0,20,0
3,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,63770,751,train,7,2,6,30.0,4,0
4,1,Chocolate Sandwich Cookies,61,19,cookies cakes,snacks,652770,764,prior,1,3,13,NaN,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33819101,49688,Fresh Foaming Cleanser,73,11,facial care,personal care,1788356,200215,prior,2,0,9,5.0,27,0
33819102,49688,Fresh Foaming Cleanser,73,11,facial care,personal care,3401313,200377,prior,1,4,11,NaN,5,0
33819103,49688,Fresh Foaming Cleanser,73,11,facial care,personal care,809510,200873,prior,5,3,8,15.0,12,0
33819104,49688,Fresh Foaming Cleanser,73,11,facial care,personal care,2359893,200873,prior,9,3,15,5.0,11,1


In [37]:
all.shape

(33819106, 15)